In [20]:
import torch
import torchvision
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample


In [56]:
# Load the data
data = np.loadtxt("datasetC.csv", delimiter = ",", dtype = np.float64 )

X = data[:,:-1]
y = data[:,-1]

# transform the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = torch.tensor(y, dtype = torch.long)

# split the data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

y_train = torch.LongTensor(y_train)

In [57]:
X = torch.tensor(X, dtype = torch.long)
X.shape

torch.Size([5000, 400])

In [58]:
# Create class iterator
class DatasetIterator(Dataset):
  # initialize function
  def __init__(self, features, labels = None):
    self.features = features
    self.labels = labels


  def __len__(self):
    return len(self.features)


  def __getitem__(self,idx):
    x = self.features[idx]
    y = self.labels[idx] if self.labels is not None else None
    return x, y


In [59]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"We are using {device}")

We are using cpu


In [60]:
class SequentialNeuralNetwork(nn.Module):
    # Initialize the Neural Network
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.2):
        super(SequentialNeuralNetwork, self).__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_stack(x)
        return output


In [122]:
def train_loop(model, train_dataloader, optimizer, loss_fn, return_loss=False):
    model.train()

    for batch, (X, y) in enumerate(train_dataloader):
        X = X.to(device)
        y = y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss_value = loss.item()
            print(f"Loss {loss_value:>4f}")


In [123]:
def test_loop(model, test_dataloader, loss_fn, return_accuracy=False):
    model.eval()

    size = len(test_dataloader.dataset)

    with torch.no_grad():
        test_loss, correct = 0, 0

        for X, y in test_dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(dim=1) == y).sum().item()

        test_loss /= size
        correct /= size

        print(f"Test Results: Accuracy: {100 * correct:.2f}%, Avg Loss: {test_loss:.6f}")



        if return_accuracy:
            return 100 * correct





# Assuming X_test and y_test are your test data
# For example, you can use them like this:




In [124]:
my_classifier = SequentialNeuralNetwork(input_dim = X_train.shape[1], hidden_dim = 512, output_dim = len(np.unique(y)))
optimizer = optim.Adam(my_classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [125]:
train_dataset = DatasetIterator(X_train, y_train)
test_dataset = DatasetIterator(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

#train_loop(my_classifier, train_dataloader, optimizer, criterion)
test_accuracy = test_loop(my_classifier, test_dataloader, criterion, return_accuracy=True)

RuntimeError: ignored

In [126]:
# perform bootstrapping
num_bootstrap_samples = 10

# Assuming you have DataLoader instances for the original data


# Create the bootstrap loop
for idx in range(num_bootstrap_samples):
    print(f"\n Bootstrap Iteration {idx + 1}")

    bootstrap_X, bootstrap_y = resample(X, y, replace=True, random_state=idx)

    # Split the bootstrap sample into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(bootstrap_X, bootstrap_y, test_size=0.2, random_state=42)
    #y_train = torch.LongTensor(y_train)

    # Create bootstrap sample DataLoader
    bootstrap_dataset = DatasetIterator(X_train, y_train)  # Use the original dataset here
    bootstrap_dataloader = DataLoader(bootstrap_dataset, batch_size=100, shuffle=True)

    # Train classifier with bootstrap DataLoader
    #train_loop(my_classifier, bootstrap_dataloader, optimizer, criterion)

    # Test classifier with original test DataLoader
    test_accuracy = test_loop(my_classifier, test_dataloader,optimizer, criterion, return_accuracy=True)
    print(f"Accuracy on original test set: {test_accuracy:.2f}%")




 Bootstrap Iteration 1


TypeError: ignored